In [4]:
#import tensorflow as tf
import numpy as np
import time
#import keras_cv
import cv2
#from tensorflow import keras
from matplotlib import pyplot as plt
import carla
from enum import Enum
from ultralytics import YOLO
#import easyocr
import paho.mqtt.client as mqtt
import pygame
from paddleocr import PaddleOCR


client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
spectator = world.get_spectator()
model_input_shape = 1024
detected_plates = {}
DETECTION_TOPIC = "plates/detection"


A_BUTTON = 0
X_BUTTON = 1
RT_BUTTON = 6
STEERING_AXIS = 0
THROTTLE_AXIS = 3
BRAKE_AXIS = 4

class LightIntensity(Enum):
    NONE = 0,
    LOW = 1,
    HIGH = 2,
    BOTH = 3

wheel = None
BUTTON_COOLDOWN = 500 # Since some inputs are activated with a button, it's usually pressed down for more than 1 tick, so it needs a cooldown to avoid multiple presses

PLATE_LENGTH = 7
MQTT_ADDRESS = "127.0.0.1"
MQTT_PORT = 1883
sender = mqtt.Client()
sender.connect(MQTT_ADDRESS, MQTT_PORT)

class CameraPosition(Enum):
    FRONT = 0
    BACK = 1

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def resize_image(image):
    if max(image.shape[0], image.shape[1]) > model_input_shape:
        if image.shape[0] > image.shape[1]:
            height = model_input_shape
            width = int(image.shape[1] * model_input_shape / image.shape[0])
        else:
            height = int(image.shape[0] * model_input_shape / image.shape[1])
            width = model_input_shape
    resized_image = cv2.resize(image, (int(width), int(height)), cv2.INTER_AREA)
    resized_image = cv2.copyMakeBorder(resized_image, 0, model_input_shape - resized_image.shape[0], 0, model_input_shape - resized_image.shape[1], cv2.BORDER_CONSTANT, value=[0, 0, 0])
    scale = model_input_shape / max(image.shape[0], image.shape[1])
    return scale, resized_image

def current_milliseconds_time():
    return int(time.time() * 1000)


def update_detected_plates(plates_list, ticks_threshold=30, n_consecutive_ticks=5):
    new_confirmed_plates = []
    removed_plates = []
    #print("Start plates list: " + str(plates_list))
    # Update or add plates in the detected list
    for plate in plates_list:
        if plate not in detected_plates.keys():
            detected_plates[plate] = {
                "ticks_since_last_detection": 0,
                "consecutive_detections": 1,
                "is_confirmed": False
            }
            #print("Probably new plate!")
        else:
            detected_plates[plate] = {
                "ticks_since_last_detection": 0,
                "consecutive_detections": int(detected_plates[plate]["consecutive_detections"]) + 1,
                "is_confirmed": detected_plates[plate]["is_confirmed"]
            }
            #detected_plates[plate]["ticks_since_last_detection"] = 0
            #detected_plates[plate]["consecutive_detections"] += 1
            #print("New plate not confirmed, but still detected! Plate: " + plate + " with consecutive detections: " + str(detected_plates[plate]["consecutive_detections"]))

        # Confirm detection after n consecutive ticks that is detected
        if (not detected_plates[plate]["is_confirmed"] and
                int(detected_plates[plate]["consecutive_detections"]) >= int(n_consecutive_ticks)):
            detected_plates[plate]["is_confirmed"] = True
            new_confirmed_plates.append(plate)
            #print(f"Plate {plate} confirmed as new.")

    # Update ticks and remove stale plates
    stale_plates = []
    for plate, data in detected_plates.items():
        if plate not in plates_list:
           # print("Plate: " + plate + "; Plates list: " + str(plates_list))
            data["ticks_since_last_detection"] += 1
            #data["consecutive_detections"] = 0  # Reset consecutive detections if not seen
            #print("COUNT RESET")
        if data["ticks_since_last_detection"] > ticks_threshold:
            stale_plates.append(plate)

    for plate in stale_plates:
        removed_plates.append(plate)
        del detected_plates[plate]

    # Generate message
    message = ""
    if new_confirmed_plates:
        message += f"New plates confirmed: {', '.join(new_confirmed_plates)}\n"
    if removed_plates:
        message += f"Plates not detected anymore: {', '.join(removed_plates)}"

    if message:
        send_mqtt_message(DETECTION_TOPIC, message)
        
# Convert raw image to numpy array to display on pygame
def image_to_pygame(image):
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))  # BGRA format
    array = array[:, :, :3]  # Keep only RGB
    array = array[:, :, ::-1]  # Convert BGR to RGB
    return array
    
def send_mqtt_message(topic, message):
    sender.publish(topic, message)

# Callback function to handle incoming messages
def on_message(client, userdata, msg):
    print(f"Received message: '{msg.payload.decode()}'")

# Callback function for connection success
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected successfully to broker")
        client.subscribe(DETECTION_TOPIC)  # Subscribe to the topic after connection
        print(f"Subscribed to topic: {DETECTION_TOPIC}")
    else:
        print(f"Connection failed with code {rc}")

def fix_plate_text(plate_text):
    # Undetectable typos: 0-O-D, 1-I-J
    # Pattern: Number - Letter Letter Letter - Number Number Number
    plate_text = plate_text.upper()
    common_typos = {
        "0": "D",
        "1": "I",
        "2": "Z",
        "4": "A",
        "5": "S",
        "7": "T",
        "8": "B",
        "]": "I",
        "[": "I",
        "O": "0",
        "D": "0",
        "I": "1",
        "Z": "2",
        "A": "4",
        "S": "5",
        "T": "7",
        "B": "8"
    }
    new_text = ""
    for i, character in enumerate(plate_text):
        if (i in [1, 2, 3] and character.isdigit()) or (i in [0, 4, 5, 6] and character.isalpha()) or character in ["[", "]"]:
            try:
                new_text += common_typos.get(character)
            except:
                #print("Tried to access key " + character + " but not found")
                new_text += "?"
        elif character.isalnum():
            new_text += character
    return new_text.upper()

def filter_boxes(boxes):
    boxes_to_remove = []
    if len(boxes) > 1:
        for i in range(len(boxes)):
            for j in range(len(boxes)):
                if i < j:
                    if intersection_over_union(boxes[i].xyxy[0], boxes[j].xyxy[0]) > 0.6:
                        boxes_to_remove.append(j)
    return boxes_to_remove

def intersection_over_union(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    unionArea = float(boxAArea + boxBArea - interArea)
    iou = interArea / unionArea
    return iou

def get_wheel_inputs():
    pygame.event.pump() 

    brake = wheel.get_axis(BRAKE_AXIS) + 1
    throttle = (wheel.get_axis(THROTTLE_AXIS) + 1) / 2
    steering = wheel.get_axis(STEERING_AXIS)
    reverse = wheel.get_button(RT_BUTTON) # 0 = A, 1 = X, 2 = B, 3 = Y, 4 = Manopola a destra in alto, 5 = manopola a sinistra in alto, 6 = RT
    quit_simulation = wheel.get_button(X_BUTTON)
    lights = wheel.get_button(A_BUTTON)

    return throttle, brake, steering, reverse, quit_simulation, lights


camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '30')
camera_bp.set_attribute('sensor_tick', '0.03')
camera_bp.set_attribute('motion_blur_intensity', '0.0')

camera_driver_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_driver_bp.set_attribute('image_size_x', '1920')
camera_driver_bp.set_attribute('image_size_y', '1080')
camera_driver_bp.set_attribute('fov', '90')
camera_driver_bp.set_attribute('sensor_tick', '0')
camera_driver_bp.set_attribute('motion_blur_intensity', '0.0')

spawn_point = carla.Transform()

#yolov8_backbone = keras_cv.models.YOLOV8Backbone.from_preset('yolo_v8_s_backbone_coco',load_weights=True)
yolov8_plates_detector = YOLO("yolo_pytorch/best.pt")

#yolov8_plates_detector = keras_cv.models.YOLOV8Detector(backbone=yolov8_backbone,num_classes=1,bounding_box_format='xywh')
#yolov8_plates_detector.compile(optimizer='adam', classification_loss='binary_crossentropy', box_loss='ciou')
#yolov8_plates_detector.load_weights("model_checkpoints_3.8/checkpoint_epoch_24.weights.h5")
#reader = easyocr.Reader(["en"], gpu=True)
paddle = PaddleOCR(lang='en')

[2025/01/30 10:49:18] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\Claudio/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\Claudio/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=N

In [9]:
mqttclient = mqtt.Client()
mqttclient.on_connect = on_connect
mqttclient.on_message = on_message
mqttclient.connect(MQTT_ADDRESS, MQTT_PORT)
# Start the network loop to process incoming messages
mqttclient.loop_start()

sender = mqtt.Client()
sender.connect(MQTT_ADDRESS, MQTT_PORT)


ego_vehicle = spawn_vehicle(vehicle_index=2, spawn_index=7)
#for i in range(10):
#    try:
#        car = spawn_vehicle(spawn_index=i)
#        car.set_autopilot(True)
#    except:
#        pass # A car already exists in the spawn point, but with many cars around it's not a problem if some does not spawn
static_vehicle = spawn_vehicle(spawn_index=9)
spawn_vehicle(spawn_index=8)
ego_vehicle_controls = carla.VehicleControl()

pygame.init()
pygame.joystick.init()

front_camera_detection = world.spawn_actor(camera_bp, carla.Transform(carla.Location(x=1.7, z=1.0), carla.Rotation(pitch=-5)), attach_to=ego_vehicle)
#back_camera_detection = world.spawn_actor(camera_bp, carla.Transform(carla.Location(x=-1.7, z=1.0), carla.Rotation(pitch=-5, yaw=-180)), attach_to=ego_vehicle)
front_camera_driver = world.spawn_actor(camera_driver_bp, carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), attach_to=ego_vehicle)

video_output = np.zeros((model_input_shape, model_input_shape, 4), dtype=np.uint8)
#back_video_output = np.zeros((model_input_shape, model_input_shape, 4), dtype=np.uint8)
#plate_area = None
driver_sight_scene = None

saved_location = carla.Location(x=0, y=0, z=0)


#Used to enhance the contrast of the read plate, improving visibility and potentially the chance to be correctly read
sharpen_kernel = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]])

def detector_callback(image, camera_position):
    global video_output
    #global back_video_output
    #global plate_area
    image_array = np.frombuffer(image.raw_data, dtype=np.uint8).reshape((image.height, image.width, 4))
    image_bgr = cv2.cvtColor(image_array, cv2.COLOR_BGRA2BGR)
    current_detected_plates = []
    scale, resized_image = resize_image(image_bgr)
    pred = yolov8_plates_detector.predict(resized_image, verbose=False)
    rescaling_factor = 1 / scale

    for result in pred:
        boxes = result.boxes
        if boxes:
            boxes_to_skip = filter_boxes(boxes)
            #print(boxes_to_remove)
            for i in range(len(boxes)):
                if i not in boxes_to_skip:
                    box = boxes[i]
                    plate = image_bgr[int(box.xyxy[0][1].item() * rescaling_factor) : int(box.xyxy[0][3].item() * rescaling_factor), int(box.xyxy[0][0].item() * rescaling_factor) : int(box.xyxy[0][2].item() * rescaling_factor)]
                    plate = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
                    plate = plate[int(plate.shape[0] * 0.3) : int(plate.shape[0] * 0.85), :]
                    plate = cv2.resize(plate, (plate.shape[1] * 2, plate.shape[0] * 2), cv2.INTER_LANCZOS4)
                    plate_enhanced = cv2.filter2D(plate, -1, sharpen_kernel)
                    #plate_reading = reader.readtext(plate_enhanced)
                    plate_reading_paddle = paddle.ocr(plate_enhanced, det=False, cls=False)
                    cv2.rectangle(image_bgr, (int(box.xyxy[0][0].item() * rescaling_factor), int(box.xyxy[0][1].item() * rescaling_factor)), (int((box.xyxy[0][2].item() * rescaling_factor)), int((box.xyxy[0][3].item() * rescaling_factor))), (0, 255, 0), 1)
                    read_flag = False
                    for plate_detected in plate_reading_paddle:
                        confidence = plate_detected[0][1]
                        plate_text = fix_plate_text(plate_detected[0][0])
                        #print("easyocr plate: " + plate_text + ", paddleocr result: " + str(plate_reading_paddle[0][0][1][0]))
                        #print(plate_text)
                        if confidence > 0.5 and len(plate_text) == PLATE_LENGTH:
                            #print(len(plate_text))
                            read_flag = True
                            cv2.putText(image_bgr, plate_text, (int(box.xyxy[0][0].item() * rescaling_factor), int(box.xyxy[0][1].item() * rescaling_factor)), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                            current_detected_plates.append(plate_text)
                    if not read_flag:
                        cv2.putText(image_bgr, "????", (int(box.xyxy[0][0].item() * rescaling_factor), int(box.xyxy[0][1].item() * rescaling_factor)), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
    update_detected_plates(current_detected_plates)
    if camera_position == CameraPosition.FRONT:
        video_output = image_bgr
    elif camera_position == CameraPosition.BACK:
        back_video_output = image_bgr

def camera_callback(image):
    global driver_sight_scene
    #driver_sight_scene = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    driver_sight_scene = image_to_pygame(image)


#Inputs for Logitech G HUB PRO Racing Wheel

front_camera_detection.listen(lambda image: detector_callback(image, CameraPosition.FRONT))
#back_camera_detection.listen(lambda image: detector_callback(image, CameraPosition.BACK))
front_camera_driver.listen(lambda image: camera_callback(image))

if pygame.joystick.get_count() == 0:
    print("Steering wheel not detected!")
else:
    wheel = pygame.joystick.Joystick(0)
    wheel.init()
    print("Steering wheel connected: " + str(wheel.get_name()))

#ego_vehicle.apply_control(ego_vehicle_controls)
#ego_vehicle.set_autopilot(True)
#static_vehicle.set_autopilot(True)

cv2.namedWindow('RGB Front Camera', cv2.WINDOW_AUTOSIZE)
#cv2.namedWindow('RGB Back Camera', cv2.WINDOW_AUTOSIZE)
#cv2.namedWindow('Plate', cv2.WINDOW_AUTOSIZE)
driver_camera = pygame.display.set_mode((1920, 1080))
pygame.display.set_caption('Driver view')
last_reverse = 0
last_toggle_lights = 0
lights_on = False
running = True
try:
    while running:
        if wheel is None:
            steering = False
            keys_held_down = pygame.key.get_pressed()
            if keys_held_down[pygame.K_w]:
                ego_vehicle_controls.throttle = min(1, ego_vehicle_controls.throttle + 0.01)
                ego_vehicle_controls.brake = 0
            if keys_held_down[pygame.K_a]:
                ego_vehicle_controls.steer = max(-1, ego_vehicle_controls.steer - 0.01)
                steering = True
            if keys_held_down[pygame.K_d]:
                ego_vehicle_controls.steer = min(1, ego_vehicle_controls.steer + 0.01)
                steering = True

            for event in pygame.event.get():
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_s:
                        ego_vehicle_controls.throttle = 0
                        ego_vehicle_controls.brake = 0.2
                    elif event.key == pygame.K_r: #Reverse
                        ego_vehicle_controls.reverse = not ego_vehicle_controls.reverse
                    elif event.key == pygame.K_0:
                        send_mqtt_message(DETECTION_TOPIC, "Message sent!")
                    elif event.key == pygame.K_q:
                        running = False
                        break
            if not steering:
                ego_vehicle_controls.steer = 0
        
        else:
            throttle, brake, steer, reverse, quit_simulation, toggle_lights = get_wheel_inputs()
        #if old_steer is None or old_steer != steer:
            #print("Throttle: " + str(throttle))
            #print("Brake: " + str(brake))
            #print(str(reverse))
            #print("Steer: " + str(steer))

                # Apply inputs to vehicle control
            if quit_simulation:
                running = False
                break
            ego_vehicle_controls.throttle = throttle
            ego_vehicle_controls.brake = brake
            ego_vehicle_controls.steer = steer
            if reverse and current_milliseconds_time() - last_reverse > BUTTON_COOLDOWN:
                ego_vehicle_controls.reverse = not ego_vehicle_controls.reverse
                last_reverse = current_milliseconds_time()
            if ego_vehicle.attributes["has_lights"] == "true" and toggle_lights and current_milliseconds_time() - last_toggle_lights > BUTTON_COOLDOWN:
                last_toggle_lights = current_milliseconds_time()
                lights_on = not lights_on
                if lights_on:
                    ego_vehicle.set_light_state(carla.VehicleLightState(int(LightIntensity.BOTH.value)))
                else:
                    ego_vehicle.set_light_state(carla.VehicleLightState(int(LightIntensity.NONE.value[0])))
        ego_vehicle.apply_control(ego_vehicle_controls)
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        if video_output is not None:
            cv2.imshow('RGB Front Camera', video_output)
        if driver_sight_scene is not None:
            pygame_image = pygame.surfarray.make_surface(driver_sight_scene.swapaxes(0, 1))  # Swap axes for pygame
            driver_camera.blit(pygame_image, (0, 0))
        pygame.display.flip()
except Exception as e:
    print(f"Error occurred: {e}")
finally:
    print("Shutting down...")
    cv2.destroyAllWindows()
    if 'front_camera_detection' in locals() and front_camera_detection.is_alive:
        front_camera_detection.destroy()
    if 'front_camera_driver' in locals() and front_camera_driver.is_alive:
        front_camera_driver.destroy()

    for actor in world.get_actors():
        if actor.type_id.startswith('vehicle.') or actor.type_id.startswith('sensor.camera'):
            actor.destroy()
    mqttclient.loop_stop()
    mqttclient.disconnect()
    pygame.quit()


Connected successfully to broker
Subscribed to topic: plates/detection
Steering wheel not detected!
Received message: 'New plates confirmed: 7VDU783
'
Received message: 'New plates confirmed: 9FUA447
'
Received message: 'Plates not detected anymore: 4NEN877, 4PKA002'
Received message: 'Plates not detected anymore: 7VOU783'
Received message: 'New plates confirmed: 9FUA441
'
Shutting down...


Shutting down...


In [ ]:
sender.disconnect()

In [6]:
for vehicle in world.get_actors().filter('vehicle.*'):
    vehicle.destroy()


In [6]:
world.set_weather(carla.WeatherParameters.ClearNight)

In [5]:
world.set_weather(carla.WeatherParameters.ClearNoon)

In [16]:
import torch, torchvision
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torchvision.__version__)


True
11.8
90100
0.19.1+cu118
